In [54]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu

ACCOUNT_BUNDLER_LIST = dataiku.Dataset("ACCOUNT_BUNDLER_LIST")
ACCOUNT_BUNDLER_LIST_df = ACCOUNT_BUNDLER_LIST.get_dataframe()
print("Account Bundlers:", len(ACCOUNT_BUNDLER_LIST_df))

#NAFCUSTOMER_RDW_CONVERSIONS = dataiku.Dataset("NAFCUSTOMER_RDW_CONVERSIONS")
#NAFCUSTOMER_RDW_CONVERSIONS_df = NAFCUSTOMER_RDW_CONVERSIONS.get_dataframe()
#print(len(NAFCUSTOMER_RDW_CONVERSIONS_df))

# Read recipe inputs
NAFCUSTOMER_ACCOUNT_ACTIVE_CARDS_BY_QUARTER = dataiku.Dataset("NAFCUSTOMER_ACCOUNT_ACTIVE_CARDS_BY_QUARTER")
NAFCUSTOMER_ACCOUNT_ACTIVE_CARDS_BY_QUARTER_df = NAFCUSTOMER_ACCOUNT_ACTIVE_CARDS_BY_QUARTER.get_dataframe()
print(len(NAFCUSTOMER_ACCOUNT_ACTIVE_CARDS_BY_QUARTER_df))

print(NAFCUSTOMER_ACCOUNT_ACTIVE_CARDS_BY_QUARTER_df.YEAR_NUMBER.min())

print("Min Year:", NAFCUSTOMER_ACCOUNT_ACTIVE_CARDS_BY_QUARTER_df.YEAR_NUMBER.min())
print("Max Year:", NAFCUSTOMER_ACCOUNT_ACTIVE_CARDS_BY_QUARTER_df.YEAR_NUMBER.max())

Account Bundlers: 274
3782998
2010
Min Year: 2010
Max Year: 2013


In [70]:
NAFCUSTOMER_ACCOUNT_ACTIVE_CARDS_BY_QUARTER_df.head()

,CUSTOMER_ACCOUNT_ID,CUSTOMER_ACCOUNT_NAME,EDW_CUSTOMER_NAME,ACCOUNT_SINCE_DATE,CUSTOMER_SOURCE_SYSTEM_CODE,ACCOUNT_OPEN_DATE,ACCOUNT_CLOSED_DATE,ATTRITION_TYPE_NAME,ATTRITION_REASON_CODE,ATTRITION_REASON_DESC,YEAR_NUMBER,QUARTER_NUMBER,ACTIVE_CARD_COUNT
0,9100000937386,GSA REGION 01,GSA FLEET,2010-06-03 00:00:00.000,SIEBEL,2010-06-03 00:00:00.000,9999-12-31 00:00:00.000,Unknown,Unkn,Unknown,2011,2,7196
1,464006984884,COMMUNITY ELECTRIC INC. (2),ANNIS PETROLEUM PRODUCTS INC,2006-03-08 00:00:00.000,SIEBEL,2006-03-08 00:00:00.000,2015-08-31 00:00:00.000,NNCL Conversion,03,Account consolidation,2013,1,20
2,7000000003987,GSA REGION 03,GSA FLEET,2008-09-27 00:00:00.000,SIEBEL,2008-09-27 00:00:00.000,9999-12-31 00:00:00.000,Unknown,Unkn,Unknown,2013,1,16399
3,447007659253,C R CONSULTING SERVICES INC.,C R CONSULTING SERVICES INC.,2008-01-09 00:00:00.000,TANDEM,2008-01-09 00:00:00.000,2022-05-16 00:00:00.000,NNCL Conversion,04,Account Conversion,2010,3,4
4,9100001408959,GREENLAND,FAIRPOINT COMMUNICATIONS,2011-10-18 00:00:00.000,SIEBEL,2011-10-18 00:00:00.000,2017-06-05 00:00:00.000,Voluntary,16,Customer Would Not Give Reason,2012,1,91


In [56]:
df_a = NAFCUSTOMER_ACCOUNT_ACTIVE_CARDS_BY_QUARTER_df

df_a.columns = ['CUSTOMER_ACCOUNT_ID','CUSTOMER_ACCOUNT_NAME','EDW_CUSTOMER_NAME','ACCOUNT_SINCE_DATE','CUSTOMER_SOURCE_SYSTEM_CODE','ACCOUNT_OPEN_DATE','ACCOUNT_CLOSED_DATE','ATTRITION_TYPE_NAME','ATTRITION_REASON_CODE','ATTRITION_REASON_DESC','YEAR_NUMBER','QUARTER_NUMBER','ACTIVE_CARD_COUNT']

df_a['CUSTOMER_ACCOUNT_ID'] = df_a['CUSTOMER_ACCOUNT_ID'].astype('Int64', errors='ignore')
df_a['CUSTOMER_ACCOUNT_NAME'] = df_a['CUSTOMER_ACCOUNT_NAME'].str.upper()
df_a['EDW_CUSTOMER_NAME'] = df_a['EDW_CUSTOMER_NAME'].str.upper()

ACCOUNT_BUNDLER_LIST_df['IS_BUNDLER'] = True
ACCOUNT_BUNDLER_LIST_df = ACCOUNT_BUNDLER_LIST_df[['EDW_CUSTOMER_NAME','IS_BUNDLER']]
ACCOUNT_BUNDLER_LIST_df.head()

df = pd.merge(df_a,ACCOUNT_BUNDLER_LIST_df, how='left', on='EDW_CUSTOMER_NAME')
df.loc[df["IS_BUNDLER"].isnull(),'IS_BUNDLER'] = False
df.head()

,CUSTOMER_ACCOUNT_ID,CUSTOMER_ACCOUNT_NAME,EDW_CUSTOMER_NAME,ACCOUNT_SINCE_DATE,CUSTOMER_SOURCE_SYSTEM_CODE,ACCOUNT_OPEN_DATE,ACCOUNT_CLOSED_DATE,ATTRITION_TYPE_NAME,ATTRITION_REASON_CODE,ATTRITION_REASON_DESC,YEAR_NUMBER,QUARTER_NUMBER,ACTIVE_CARD_COUNT,IS_BUNDLER
0,9100000937386,GSA REGION 01,GSA FLEET,2010-06-03 00:00:00.000,SIEBEL,2010-06-03 00:00:00.000,9999-12-31 00:00:00.000,Unknown,Unkn,Unknown,2011,2,7196,False
1,464006984884,COMMUNITY ELECTRIC INC. (2),ANNIS PETROLEUM PRODUCTS INC,2006-03-08 00:00:00.000,SIEBEL,2006-03-08 00:00:00.000,2015-08-31 00:00:00.000,NNCL Conversion,03,Account consolidation,2013,1,20,False
2,7000000003987,GSA REGION 03,GSA FLEET,2008-09-27 00:00:00.000,SIEBEL,2008-09-27 00:00:00.000,9999-12-31 00:00:00.000,Unknown,Unkn,Unknown,2013,1,16399,False
3,447007659253,C R CONSULTING SERVICES INC.,C R CONSULTING SERVICES INC.,2008-01-09 00:00:00.000,TANDEM,2008-01-09 00:00:00.000,2022-05-16 00:00:00.000,NNCL Conversion,04,Account Conversion,2010,3,4,False
4,9100001408959,GREENLAND,FAIRPOINT COMMUNICATIONS,2011-10-18 00:00:00.000,SIEBEL,2011-10-18 00:00:00.000,2017-06-05 00:00:00.000,Voluntary,16,Customer Would Not Give Reason,2012,1,91,False


In [57]:
df['CUSTOMER'] = np.nan
df['CUST_CALC_SOURCE'] = 'Unknown'
df.loc[df.IS_BUNDLER,'EDW_CUSTOMER_NAME'] = np.nan

df.loc[~df['EDW_CUSTOMER_NAME'].isnull(),'CUSTOMER'] = df["EDW_CUSTOMER_NAME"]
df.loc[~df['EDW_CUSTOMER_NAME'].isnull(),'CUST_CALC_SOURCE'] = 'EDW'

df.loc[df['CUSTOMER'].isnull(),'CUST_CALC_SOURCE'] = 'ACCOUNT'
df.loc[df['CUSTOMER'].isnull(),'CUSTOMER'] = df.CUSTOMER_ACCOUNT_NAME


ending_tokens = [' 2', ' 3', ' 4', ' 04', ' 5', ' 6', ' 7', ' 8', ' 9',' (2)',
                 ' (3)',' (04)',' (4)', ' (5)', ' (6)', ' (7)', ' (8)',
                 ' (9)',' (25)','  (32)', ' AD', ' LD', 'L1']

df['CUSTOMER'].str.strip()

for s in ending_tokens:
    index_offset = -1*(len(s))
    df.loc[df['CUSTOMER'].str.endswith(s, na=False),"CUSTOMER"] = df['CUSTOMER'].str[:index_offset]


In [61]:
# Convert YEAR AND QUARTER INTO DATE REPRESENTING THE FIRST DAY OF THE QUARTER
df['REVENUE_DATE'] = ((3*df.QUARTER_NUMBER)-2).astype(str) + "/1/" + (df.YEAR_NUMBER).astype(str)
df['REVENUE_DATE'] = pd.to_datetime(df["REVENUE_DATE"])
df['ACCOUNT_SINCE_DATE'] = pd.to_datetime(df['ACCOUNT_SINCE_DATE'])

#df['CUSTOMER STATE']
#df.loc[df['CUSTOMER_ACCOUNT_NAME'].str.endswith(s, na=False),"CUSTOMER_ACCOUNT_NAME"] = df['CUSTOMER_ACCOUNT_NAME'].str[:index_offset]

df.head()

,CUSTOMER_ACCOUNT_ID,CUSTOMER_ACCOUNT_NAME,EDW_CUSTOMER_NAME,ACCOUNT_SINCE_DATE,CUSTOMER_SOURCE_SYSTEM_CODE,ACCOUNT_OPEN_DATE,ACCOUNT_CLOSED_DATE,ATTRITION_TYPE_NAME,ATTRITION_REASON_CODE,ATTRITION_REASON_DESC,YEAR_NUMBER,QUARTER_NUMBER,ACTIVE_CARD_COUNT,IS_BUNDLER,CUSTOMER,CUST_CALC_SOURCE,REVENUE_DATE
0,9100000937386,GSA REGION 01,GSA FLEET,2010-06-03,SIEBEL,2010-06-03 00:00:00.000,9999-12-31 00:00:00.000,Unknown,Unkn,Unknown,2011,2,7196,False,GSA FLEET,EDW,2011-04-01
1,464006984884,COMMUNITY ELECTRIC INC. (2),ANNIS PETROLEUM PRODUCTS INC,2006-03-08,SIEBEL,2006-03-08 00:00:00.000,2015-08-31 00:00:00.000,NNCL Conversion,03,Account consolidation,2013,1,20,False,ANNIS PETROLEUM PRODUCTS INC,EDW,2013-01-01
2,7000000003987,GSA REGION 03,GSA FLEET,2008-09-27,SIEBEL,2008-09-27 00:00:00.000,9999-12-31 00:00:00.000,Unknown,Unkn,Unknown,2013,1,16399,False,GSA FLEET,EDW,2013-01-01
3,447007659253,C R CONSULTING SERVICES INC.,C R CONSULTING SERVICES INC.,2008-01-09,TANDEM,2008-01-09 00:00:00.000,2022-05-16 00:00:00.000,NNCL Conversion,04,Account Conversion,2010,3,4,False,C R CONSULTING SERVICES INC.,EDW,2010-07-01
4,9100001408959,GREENLAND,FAIRPOINT COMMUNICATIONS,2011-10-18,SIEBEL,2011-10-18 00:00:00.000,2017-06-05 00:00:00.000,Voluntary,16,Customer Would Not Give Reason,2012,1,91,False,FAIRPOINT COMMUNICATIONS,EDW,2012-01-01


In [62]:
df.CUST_CALC_SOURCE.value_counts(dropna=False)

EDW        3442480
ACCOUNT     340518
Name: CUST_CALC_SOURCE, dtype: int64

In [69]:
df_cust_since = df.groupby(['CUSTOMER']).ACCOUNT_SINCE_DATE.min().reset_index()
df_cust_since = df_cust_since.sort_values(by=['CUSTOMER'], ascending=True)
df_cust_since.head(10)

,CUSTOMER,ACCOUNT_SINCE_DATE
0,1 AUTO PARTS INC,2012-10-29
1,D F W STONE SUPPLY,2012-06-19
2,HISTORIC HOTELS LTD,2010-02-22
3,K C ELECTRIC LLC,2010-12-21
4,MBI ENERGY SERVICES,2013-04-05
5,PRINTING SOLUTIONS INC,2011-03-25
6,PRO CUT,2012-03-27
7,#1 COOLING & HEATING LLC,2012-12-21
8,#1 FISH MARKET INC,1999-11-11
9,'KOM LIMO,2002-09-09


In [72]:
df_cust_max_revenue = df.groupby(['CUSTOMER']).REVENUE_DATE.max().reset_index()
df_cust_max_revenue.columns = ['CUSTOMER','MAX_REVENUE_DATE']
df_cust_max_revenue = df_cust_max_revenue.sort_values(by=['CUSTOMER'], ascending=True)
df_cust_max_revenue.head(10)

,CUSTOMER,MAX_REVENUE_DATE
0,1 AUTO PARTS INC,2013-04-01
1,D F W STONE SUPPLY,2013-04-01
2,HISTORIC HOTELS LTD,2013-04-01
3,K C ELECTRIC LLC,2013-01-01
4,MBI ENERGY SERVICES,2013-04-01
5,PRINTING SOLUTIONS INC,2011-10-01
6,PRO CUT,2013-04-01
7,#1 COOLING & HEATING LLC,2013-04-01
8,#1 FISH MARKET INC,2010-10-01
9,'KOM LIMO,2013-04-01


In [73]:
df_cust_min_revenue = df.groupby(['CUSTOMER']).REVENUE_DATE.min().reset_index()
df_cust_min_revenue.columns = ['CUSTOMER','MIN_REVENUE_DATE']
df_cust_min_revenue = df_cust_min_revenue.sort_values(by=['CUSTOMER'], ascending=True)
df_cust_min_revenue.head(10)

,CUSTOMER,MIN_REVENUE_DATE
0,1 AUTO PARTS INC,2012-10-01
1,D F W STONE SUPPLY,2012-07-01
2,HISTORIC HOTELS LTD,2012-04-01
3,K C ELECTRIC LLC,2010-10-01
4,MBI ENERGY SERVICES,2013-04-01
5,PRINTING SOLUTIONS INC,2011-04-01
6,PRO CUT,2012-04-01
7,#1 COOLING & HEATING LLC,2013-01-01
8,#1 FISH MARKET INC,2010-10-01
9,'KOM LIMO,2010-01-01


In [32]:
#df_account_with_customer = df[['CUSTOMER_ACCOUNT_ID','CUSTOMER']].copy()
#df_account_with_customer = df_account_with_customer.drop_duplicates(subset='CUSTOMER_ACCOUNT_ID')
#print(len(df_account_with_customer))
#df_account_with_customer.head()

#df_g.columns = ['CUSTOMER','ACCOUNT_COUNT']
#df_g = df_g.sort_values(by=['ACCOUNT_COUNT'], ascending=False)
#df_g.head(100)

#NAFCUSTOMER_RDW_CONVERSIONS_df.head()
#NAFCUSTOMER_RDW_CONVERSIONS_df.STATUS_DATE.min()

#NAFCUSTOMER_RDW_CONVERSIONS_df.head()
#df_conv = NAFCUSTOMER_RDW_CONVERSIONS_df[['FLEET_ID','CLASSIC_ACCOUNT_NUMBER','FLEET_NAME']].copy()
#print(len(df_conv))
#df_conv = df_conv[~df_conv.CLASSIC_ACCOUNT_NUMBER.isnull()]
#print(len(df_conv))

#df_conv.columns = ['CUSTOMER_ACCOUNT_ID', 'CLASSIC_CUSTOMER_ACCOUNT_ID', 'FLEET_NAME']
#df_conv['CUSTOMER_ACCOUNT_ID'] = df_conv['CUSTOMER_ACCOUNT_ID'].astype('Int64', errors='ignore')
#df_conv['CLASSIC_CUSTOMER_ACCOUNT_ID'] = pd.to_numeric(df_conv['CLASSIC_CUSTOMER_ACCOUNT_ID'], errors='coerce')
#df_conv = df_conv[~df_conv.CLASSIC_CUSTOMER_ACCOUNT_ID.isnull()]
#df_conv['CLASSIC_CUSTOMER_ACCOUNT_ID'] = df_conv['CLASSIC_CUSTOMER_ACCOUNT_ID'].astype('int64', errors='ignore')
#df_conv['CLASSIC_CUSTOMER_ACCOUNT_ID'] = df_conv['CLASSIC_CUSTOMER_ACCOUNT_ID'].astype('Int64', errors='ignore')

#df_conv = pd.merge(df_conv, df_account_with_customer, on='CUSTOMER_ACCOUNT_ID', how='inner')
#print(len(df_conv))
#df_conv = df_conv[['CLASSIC_CUSTOMER_ACCOUNT_ID','CUSTOMER']]
#df_conv.columns = ['CLASSIC_CUSTOMER_ACCOUNT_ID','CUSTOMER_CONVERTED_TO']
#df_conv.head()

In [33]:
#df = df[['CUSTOMER_ACCOUNT_ID','CUSTOMER_ACCOUNT_NAME','YEAR_NUMBER','QUARTER_NUMBER','ACTIVE_CARD_COUNT','CUSTOMER','CUST_CALC_SOURCE']]
#print(len(df))

In [34]:
#print(len(df))
#df = pd.merge(df, df_conv, left_on='CUSTOMER_ACCOUNT_ID', right_on='CLASSIC_CUSTOMER_ACCOUNT_ID', how='left')
#print(len(df))

In [35]:
#df.CUSTOMER_CONVERTED_TO.value_counts()

In [108]:
#df.loc[~df['CUSTOMER_CONVERTED_TO'].isnull(),'CUSTOMER'] = df.CUSTOMER_CONVERTED_TO

In [36]:
#df.head()

In [37]:
# Write recipe outputs
#NAFCUSTOMER_LONG_HISTORY_MATCHES = dataiku.Dataset("NAFCUSTOMER_LONG_HISTORY_MATCHES")
#NAFCUSTOMER_LONG_HISTORY_MATCHES.write_with_schema(NAFCUSTOMER_LONG_HISTORY_MATCHES_df)